<a href="https://colab.research.google.com/github/fatisepah/samples/blob/main/implementation_just_conv_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import numpy as np

In [79]:
class Convolution2D:

    def __init__(self, inputs_channel, num_filters, kernel_size, padding, stride):
        # weight size: (F, C, K, K)
        # bias size: (F) 
        self.F = num_filters
        self.K = kernel_size
        self.C = inputs_channel

        self.weights = np.zeros((self.F, self.C, self.K, self.K))
        self.bias = np.zeros((self.F, 1))
        for i in range(0,self.F):
            self.weights[i,:,:,:] = np.random.normal(loc=0, scale=np.sqrt(1./(self.C*self.K*self.K)), size=(self.C, self.K, self.K))

        self.p = padding
        self.s = stride

        print("F(num_filters)ConvLayer=",self.F)
        print("K(kernel_size)ConvLayer=",self.K)
        print("C(inputs_channel)ConvLayer=",self.C)

    def zero_padding(self, inputs, size):
        w, h = inputs.shape[0], inputs.shape[1]
        new_w = 2 * size + w
        new_h = 2 * size + h
        out = np.zeros((new_w, new_h))
        out[size:w+size, size:h+size] = inputs
        
        return out

    def forward(self, inputs):
        # input size: (C, W, H)
        # output size: (N, F ,WW, HH)
        # print("inputOfConvLayer=",inputs)
        print("inputShapeOfConvLayer=",inputs.shape)

        C = inputs.shape[0]
        print("C-inputs.shape[0]=",C)

        W = inputs.shape[1]+2*self.p
        print("W-inputs.shape[1]=",W)

        H = inputs.shape[2]+2*self.p
        print("H-inputs.shape[2]=",H)

        self.inputs = np.zeros((C, W, H))
        # print("self.inputsConv=",self.inputs)

        for c in range(inputs.shape[0]):
            self.inputs[c,:,:] = self.zero_padding(inputs[c,:,:], self.p)
        # print("OutZero_padding=",self.inputs)

        #############  
        WW =int((W - self.K)/self.s + 1)
        print("WW=",WW)
        ##########
        HH = int((H - self.K)/self.s + 1)
        print("HH=",HH)

        feature_maps = np.zeros((self.F, WW, HH))
        # print("feature_mapsOfConvLayerzero=",feature_maps)

        for f in range(self.F):
            for w in range(0, WW, self.s):
                for h in range(0, HH, self.s):
                    # print("SelectInputh=",self.inputs[:,w:w+self.K,h:h+self.K])
                    sel=self.inputs[:,w:w+self.K,h:h+self.K]
                    # print("SelectInputShape=",sel.shape)
                    Weights=self.weights[f,:,:,:]
                    # print("WeightShape=",Weights.shape)
                    feature_maps[f,w,h]=np.sum(self.inputs[:,w:w+self.K,h:h+self.K]*self.weights[f,:,:,:])+self.bias[f]
                    # print("feature_mapsOfConvLayer=",feature_maps)
        # print("Lastfeature_mapsOfConvLayer=",feature_maps)

        return feature_maps
    

    def backward(self, dy):

        C, W, H = self.inputs.shape
        print("inputs.shape=",self.inputs.shape)


        dx = np.zeros(self.inputs.shape)
        dw = np.zeros(self.weights.shape)
        db = np.zeros(self.bias.shape)

        F, W, H = dy.shape
        for f in range(F):
            for w in range(0, W, self.s):
                for h in range(0, H, self.s):
                    dw[f,:,:,:]+=dy[f,w,h]*self.inputs[:,w:w+self.K,h:h+self.K]
                    dx[:,w:w+self.K,h:h+self.K]+=dy[f,w,h]*self.weights[f,:,:,:]

        for f in range(F):
            db[f] = np.sum(dy[f, :, :])

        self.weights -= self.lr * dw
        self.bias -= self.lr * db
        return dx

    def extract(self):
        return {self.name+'.weights':self.weights, self.name+'.bias':self.bias}

    def feed(self, weights, bias):
        self.weights = weights
        self.bias = bias

In [80]:
# bonus
class SoftmaxLayer:
    def __init__(self, input_size):
        self.input_size = input_size

        print("input_sizeOfSoftmax=",input_size)
        
    
    def forward(self, input):
        self.input = input
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)

        print("output_sizeOfSoftmaxForward=",self.output)
        return self.output
    
    def backward(self, output_error, learning_rate):
        input_error = np.zeros(output_error.shape)
        out = np.tile(self.output.T, self.input_size)
        return self.output * np.dot(output_error, np.identity(self.input_size) - out)

In [81]:
class FCLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(input_size, output_size) / np.sqrt(input_size + output_size)
        self.bias = np.random.randn(1, output_size) / np.sqrt(input_size + output_size)

        print("input_sizeOfFCLayer=",input_size)
        print("output_sizeOfFCLayer=",output_size)


    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

    def backward(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # bias_error = output_error
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [82]:
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward(self, input):
        self.input = input
        return self.activation(input)
    
    def backward(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)

In [83]:
# bonus
class FlattenLayer:
    def __init__(self, input_shape):
        self.input_shape = input_shape

        print("input_shapeOfFlatten=",input_shape)

    def forward(self, input):
        return np.reshape(input, (1, -1))
    
    def backward(self, output_error, learning_rate):
        return np.reshape(output_error, self.input_shape)

In [84]:
class Flatten:
    def __init__(self):
        pass
    def forward(self, inputs):
        self.C, self.W, self.H = inputs.shape
        return inputs.reshape(1, self.C*self.W*self.H)
    def backward(self, dy):
        return dy.reshape(self.C, self.W, self.H)
    def extract(self):
        return

In [85]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

In [86]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_pred.size

def sse(y_true, y_pred):
    return 0.5 * np.sum(np.power(y_true - y_pred, 2))

def sse_prime(y_true, y_pred):
    return y_pred - y_true

In [87]:
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_train /= 255
y_train = np_utils.to_categorical(y_train)
x_train = x_train[0:1000]
y_train = y_train[0:1000]

x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

In [88]:

from PIL import Image
# unlike the Medium article, I am not encapsulating this process in a separate class
# I think it is nice just like this
network = [
    
    Convolution2D(1, 6, 5, 0, 1),
    ActivationLayer(relu, relu_prime),

]

epochs = 40
learning_rate = 0.1

# training
for epoch in range(epochs):
    error = 0
    for x, y_true in zip(x_train, y_train):
        # forward

        # print("x=",x)
        print("y_true=",y_true)
        print("shapex=",x.shape)
        output = x.reshape((1,x.shape[0], x.shape[1]))
        # print("xReshape=",output)
        print("shapexReshape=",output.shape)
        # data = Image.fromarray(output)
        # data.show()

        for layer in network:
            output = layer.forward(output)
            
            #print("outputOfLayer=",output)
            # data = Image.fromarray(output)
            # data.show()
        
        # error (display purpose only)
        error += mse(y_true, output)

        # backward
        output_error = mse_prime(y_true, output)
        for layer in reversed(network):
            output_error = layer.backward(output_error, learning_rate)
    
    error /= len(x_train)
    print('%d/%d, error=%f' % (epoch + 1, epochs, error))

F(num_filters)ConvLayer= 6
K(kernel_size)ConvLayer= 5
C(inputs_channel)ConvLayer= 1
y_true= [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
shapex= (28, 28)
shapexReshape= (1, 28, 28)
inputShapeOfConvLayer= (1, 28, 28)
C-inputs.shape[0]= 1
W-inputs.shape[1]= 28
H-inputs.shape[2]= 28
WW= 24
HH= 24


ValueError: ignored

In [ ]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

ratio = sum([np.argmax(y) == np.argmax(predict(network, x)) for x, y in zip(x_test, y_test)]) / len(x_test)
error = sum([mse(y, predict(network, x)) for x, y in zip(x_test, y_test)]) / len(x_test)
print('ratio: %.2f' % ratio)
print('mse: %.4f' % error)

In [ ]:
import matplotlib.pyplot as plt

samples = 10
for test, true in zip(x_test[:samples], y_test[:samples]):
    image = np.reshape(test, (28, 28))
    plt.imshow(image, cmap='binary')
    plt.show()
    pred = predict(network, test)[0]
    idx = np.argmax(pred)
    idx_true = np.argmax(true)
    print('pred: %s, prob: %.2f, true: %d' % (idx, pred[idx], idx_true))